## Objective
This is a fully integrated prototype of a multi-agent system that supports the following:

1.   Conversational interface with limited memory
2.   Document-based Question Answering using RAG
3.   Text-to-image generation with prompt engineering
4.   Multi-agent task handling using a controller (Weather, SQL, Recommender)



### Part 1:

Part 1 demonstrates creating a RAG pipeline. Following steps are included:
1. Loading a .pdf file
2. Use embedding model and LLM from OpenAI
3. Retriveing response from FAISS vector database
4. Augmenting and generating the response

In [7]:
!pip install langchain.openai
!pip install langchain_classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.3 MB/s eta 0:00:00


In [8]:
# import libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
#from langchain.memory import ConversationBufferMemory
from langchain_classic.memory import ConversationBufferMemory

In [9]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [10]:
# call the model
llm = ChatOpenAI(model="gpt-4o", api_key=openai_api_key)

In [58]:
# define memory to maintain context throughout human interaction
memory = ConversationBufferMemory(
    # key under which conversation history will be stored
    memory_key="user_chat_history",
    return_messages=True
    )

In [59]:
# create a prompt  from a list of messages. Each message is a tuple where the
#  first element is the message type (e.g., "system", "user", "placeholder") and
# the second element is the content
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assisteant. Use tools if necessary to answer user query. Since you are storing chat history, every time you generate a response, start by saying 'Referring to chat history' or 'Not referring to chat history' depending upon whether or not you are using chat history to generate a response and start the response on the next line"),
    # placeholder that will be filled with the conversation history. The variable_name="chat_history"
    # means that the memory component (defined earlier as memory) will inject the past conversation
    # turns into the prompt at this point.
    MessagesPlaceholder(variable_name="user_chat_history"),
    ("user", "{user_input}"),
    # The agent_scratchpad is where the agent internally keeps track of its thoughts,
    # the tools it decided to use, the observations it got back from tools, and its
    # planning steps before generating a final response.
    ("placeholder", "{agent_scratchpad}")
]
)

In [60]:
# tool definition
from datetime import datetime, timezone
@tool
def get_utc_datetime():
  """ this function returns the current UTC date and time"""
  current_datetime = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")
  print(type(current_datetime))
  return f"Current UTC datetime is: {current_datetime}"
#get_utc_datetime()

tools = [get_utc_datetime]

In [61]:
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# wrap the agent in executor
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [63]:
response = agent_executor.invoke({"user_input": "explain machine learning in five lines"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...
Not referring to chat history

Machine learning is an area of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. Instead of being explicitly programmed to perform a task, these systems improve their performance through experience and data analysis. The core components of machine learning include algorithms, models, and training data. It can be used for various applications like classification, regression, clustering, and reinforcement learning. The ultimate goal is to enable systems to automatically learn and adapt without human intervention.

> Finished chain.
Not referring to chat history

Machine learning is an area of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. Instead of being explicitly programmed to perform a task, these systems improve their performance through experience and data analysis. The core components of 

In [64]:
response = agent_executor.invoke({"user_input": "what is current time in Singapore"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `get_utc_datetime` with `{}`


<class 'str'>
Current UTC datetime is: 2026-01-23 10:03:33Referring to chat history

Singapore is 8 hours ahead of UTC. Given the current UTC time is 10:03:33, the current time in Singapore is 18:03:33.

> Finished chain.
Referring to chat history

Singapore is 8 hours ahead of UTC. Given the current UTC time is 10:03:33, the current time in Singapore is 18:03:33.


In [65]:
response = agent_executor.invoke({"user_input": "my name is SB"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...
Not referring to chat history

Hello SB! How can I assist you today?

> Finished chain.
Not referring to chat history

Hello SB! How can I assist you today?


In [66]:
response = agent_executor.invoke({"user_input": "where is Singapore located?"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...
Not referring to chat history

Singapore is a city-state and island country located in Southeast Asia. It lies at the southern tip of the Malay Peninsula, about 137 kilometers (85 miles) north of the equator. It is bordered by the Strait of Malacca to the west, the Riau Islands of Indonesia to the south, and the South China Sea to the east. Singapore is known for its strategic location as a major hub for international trade and travel.

> Finished chain.
Not referring to chat history

Singapore is a city-state and island country located in Southeast Asia. It lies at the southern tip of the Malay Peninsula, about 137 kilometers (85 miles) north of the equator. It is bordered by the Strait of Malacca to the west, the Riau Islands of Indonesia to the south, and the South China Sea to the east. Singapore is known for its strategic location as a major hub for international trade and travel.


In [67]:
response = agent_executor.invoke({"user_input": "do you recall my name?"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...
Referring to chat history

Yes, you mentioned that your name is SB. How can I assist you further, SB?

> Finished chain.
Referring to chat history

Yes, you mentioned that your name is SB. How can I assist you further, SB?


In [76]:
# print chat history
print("Chat History")
for message in memory.load_memory_variables({}).get("user_chat_history"):
  print(message.content, "\n\n")

Chat History
explain machine learning in five lines 


Not referring to chat history

Machine learning is an area of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. Instead of being explicitly programmed to perform a task, these systems improve their performance through experience and data analysis. The core components of machine learning include algorithms, models, and training data. It can be used for various applications like classification, regression, clustering, and reinforcement learning. The ultimate goal is to enable systems to automatically learn and adapt without human intervention. 


what is current time in Singapore 


Referring to chat history

Singapore is 8 hours ahead of UTC. Given the current UTC time is 10:03:33, the current time in Singapore is 18:03:33. 


my name is SB 


Not referring to chat history

Hello SB! How can I assist you today? 


where is Singapore located? 


Not referring to chat histor

In [1]:
!pip install langchain_openai
!pip install langchain_community
!pip install langchain_core
#!pip install lan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
#!pip install --upgrade langchain

In [2]:
# Force-reinstall langchain to resolve persistent ModuleNotFoundError for langchain.memory
#!pip install --upgrade --force-reinstall langchain

# import necessary packages
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
#from langchain.memory import ConversationBufferMemory
#from langchain_core.memory import BaseMemory
#from langchain_community.memory import ConversationBufferMemory
from langchain_classic.memory import ConversationBufferMemory

In [3]:
# read the OpenAI API key from environment variable
from google.colab import userdata
openai_api_key = userdata.get("OPENAI_API_KEY").strip()

In [4]:
!pip install pypdf
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 82.5 MB/s eta 0:00:00


In [6]:
# load the pdf file
file_path = "company_policy.pdf"
loader = PyMuPDFLoader(file_path)
documents = loader.load()
#documents

In [7]:
# print number of documents loaded (24 pages in the pdf document)
print("Number of documents loaded: ", len(documents))

Number of documents loaded:  24


In [8]:
documents[0]

Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2020-08-26T06:56:00+00:00', 'source': 'company_policy.pdf', 'file_path': 'company_policy.pdf', 'total_pages': 24, 'format': 'PDF 1.5', 'title': '', 'author': 'hr', 'subject': '', 'keywords': '', 'moddate': '2020-08-26T06:56:00+00:00', 'trapped': '', 'modDate': 'D:20200826065600Z', 'creationDate': "D:20200826065600+00'00'", 'page': 0}, page_content='SPIL Corporate HR Policies  \n \n \nSIRCA PAINTS INDIA LTD \nNEW DELHI  \n \n \n \n \nCORPORATE  \n  HUMAN RESOURCES \nPOLICIES & MANUALS')

In [9]:
documents[0].page_content

'SPIL Corporate HR Policies  \n \n \nSIRCA PAINTS INDIA LTD \nNEW DELHI  \n \n \n \n \nCORPORATE  \n  HUMAN RESOURCES \nPOLICIES & MANUALS'

In [10]:
# divide the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
document_chunks = text_splitter.split_documents(documents)
print("Number of chunks: ", len(document_chunks))

# print the third, fifth and seventh chunk
print("\n**************Third chunk:**************\n")
print(document_chunks[3])
print("\n**************Fifth chunk:**************\n")
print(document_chunks[5])
print("\n**************Seventh chunk:**************\n")
print(document_chunks[7])

Number of chunks:  122

**************Third chunk:**************

page_content='Resources Department of SPIL.  
 
Applicability  
 
This EHB will be applicable to the employees working in Sirca Paints India Limited (SPIL) w.e.f 
August 21, 2020. This book contains all the notices/circulars/extracts/meetings circulated earlier 
before the date of validity of this handbook.  
 
Definitions  
 
a) “Company” means Sirca Paints India Limited (SPIL) and will its branches, offices/plants located 
anywhere in India or Abroad.' metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2020-08-26T06:56:00+00:00', 'source': 'company_policy.pdf', 'file_path': 'company_policy.pdf', 'total_pages': 24, 'format': 'PDF 1.5', 'title': '', 'author': 'hr', 'subject': '', 'keywords': '', 'moddate': '2020-08-26T06:56:00+00:00', 'trapped': '', 'modDate': 'D:20200826065600Z', 'creationDate': "D:20200826065600+00'00'", 'page': 1}

**************Fifth chunk:**************

pa

In [11]:
# create embeddings for the chunks
embeddings = OpenAIEmbeddings(api_key=openai_api_key)

# print the model used for embeddings
print(embeddings.model)

text-embedding-ada-002


In [12]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 90.2 MB/s eta 0:00:00


In [13]:
# create FAISS vector store from the list of `Document` object and embeddings
vector_store=FAISS.from_documents(document_chunks, embeddings)
# save the faiss index to disk
vector_store.save_local("fais_index")

In [14]:
# memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

/tmp/ipython-input-2010414609.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")


In [15]:
# initialize GPT4 turbo via langchain
llm = ChatOpenAI(model="gpt-4-turbo", api_key=openai_api_key.strip())

In [ ]:
!pip install langchain

In [16]:
from langchain_classic.chains import RetrievalQA
from langchain_classic.chains import ConversationalRetrievalChain

In [17]:
# create a ConversationalRetrievalChain chain
# as_retriever() converts vector store into a retriever object
retriever = vector_store.as_retriever(search_kwargs={"k":3})
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    retriever=retriever,
    return_source_documents=True,
)

In [18]:
query1 = "What does the policy say about attendance and timing?"
result1 = qa_chain({"question":query1})
print("Question 1:\n")
print(f'Human: {query1}')
print(f'AI: {result1["answer"]}')

query2 = "Can they come late?"
result2 = qa_chain({"question":query2})
print("Question 2:\n")
print(f'Human: {query2}')
print(f'AI: {result2["answer"]}')

query3 = "To whom can they submit attendance reporting?"
result3 = qa_chain({"question":query3})
print("Question 3:\n")
print(f'Human: {query3}')
print(f'AI: {result3["answer"]}')

/tmp/ipython-input-2390701118.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result1 = qa_chain({"question":query1})


Question 1:

Human: What does the policy say about attendance and timing?
AI: The policy outlines that all employees are expected to adhere to flexible working hours, with the options being from 9:30 AM to 6:00 PM or 10:00 AM to 6:30 PM. Attendance is a crucial aspect of the policy, emphasizing the principle of "NO WORK NO PAY." Unauthorized absences will be treated as "LOSS OF PAY." Additionally, employees are responsible for ensuring their attendance is properly recorded and communicated, particularly in the Marketing Department where a "Daily Time Report" (DTR) must be submitted to Reporting Officers for attendance marking.
Question 2:

Human: Can they come late?
AI: Yes, according to the attendance policy, employees are allowed to arrive late up to 11:45 AM under extreme emergency situations. Additionally, there is a 15-minute grace period for regular late arrivals. However, arriving beyond these conditions could result in leave deductions.
Question 3:

Human: To whom can they subm

In [19]:
# perform similarity search
search_query = "What is the notice period?"
similar_docs = vector_store.similarity_search(search_query, k=3)

# print the results
for i, doc in enumerate(similar_docs):
  print(f"\n**********Page Content: Document {i+1}**********\n")
  print(doc.page_content)
  print(f"Metadata: {doc.metadata}")


**********Page Content: Document 1**********

SPIL Corporate HR Policies  
 
 
g) 
For Transferee, the clause of employment, policies and procedures of new company will be 
applicable. The proper appointment letter and other joining material will be given.  
Section 10:  Notices Period 
a) 
On Probation, the employee can resigns from the services by giving 10 days of notice.  
 
b) 
On Confirmation, the employee can resign from the services by giving the notice of 30 days 
(i.e. one month).  
 
c)
Metadata: {'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2020-08-26T06:56:00+00:00', 'source': 'company_policy.pdf', 'file_path': 'company_policy.pdf', 'total_pages': 24, 'format': 'PDF 1.5', 'title': '', 'author': 'hr', 'subject': '', 'keywords': '', 'moddate': '2020-08-26T06:56:00+00:00', 'trapped': '', 'modDate': 'D:20200826065600Z', 'creationDate': "D:20200826065600+00'00'", 'page': 11}

**********Page Content: Document 2**********

month notice is t